In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime

# 나머지는 변수로 관리
label_table = "kind"
return_table = "abnormal_return_kind"
label_col = "label"

# 포함할 abnormal return 컬럼
abn_return_cols = [
    "abn_ret_minus_10m", "abn_ret_minus_9m", "abn_ret_minus_8m", "abn_ret_minus_7m", "abn_ret_minus_6m",
    "abn_ret_minus_5m", "abn_ret_minus_4m", "abn_ret_minus_3m", "abn_ret_minus_2m", "abn_ret_minus_1m",
    "abn_ret_1m", "abn_ret_2m", "abn_ret_3m", "abn_ret_4m", "abn_ret_5m", "abn_ret_6m",
    "abn_ret_7m", "abn_ret_8m", "abn_ret_9m", "abn_ret_10m"
]

# 포함할 volume 컬럼
volume_cols = [
    "volume_minus_10m", "volume_minus_9m", "volume_minus_8m", "volume_minus_7m", "volume_minus_6m",
    "volume_minus_5m", "volume_minus_4m", "volume_minus_3m", "volume_minus_2m", "volume_minus_1m",
    "volume_1m", "volume_2m", "volume_3m", "volume_4m", "volume_5m", "volume_6m",
    "volume_7m", "volume_8m", "volume_9m", "volume_10m"
]

# SQL 컬럼 문자열을 만듬 (event_id, event_ts는 하드코딩)
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"ar.{col}" for col in volume_cols] +
    [f"k.{label_col}"]
)
sql_columns_str = ",\n    ".join(sql_columns)

query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
ORDER BY ar.event_ts ASC
"""

df_total = pd.read_sql(query, engine)
df_total["period_dummy"] = (
    pd.to_datetime(df_total["event_ts"], utc=True) > pd.Timestamp("2022-06-30", tz="UTC")
).astype(int)
df_total

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_minus_10m,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,...,volume_3m,volume_4m,volume_5m,volume_6m,volume_7m,volume_8m,volume_9m,volume_10m,label,period_dummy
0,2021-01-04 10:08:00+00:00,-2.14,-2.14,-1.20,-1.32,-1.93,-2.38,-2.05,-2.45,-1.76,...,738.0,3081.0,6644.0,3178.0,1059.0,1373.0,1244.0,1457.0,0,0
1,2021-01-04 10:08:00+00:00,NaN,NaN,-1.45,-1.64,-1.44,-2.24,-2.49,-2.69,-1.96,...,11510.0,2956.0,3751.0,1782.0,4513.0,337.0,34.0,1704.0,1,0
2,2021-01-04 10:08:00+00:00,-2.14,-2.14,-1.20,-1.32,-1.93,-2.38,-2.05,-2.45,-1.76,...,738.0,3081.0,6644.0,3178.0,1059.0,1373.0,1244.0,1457.0,0,0
3,2021-01-04 10:14:00+00:00,-8.40,-8.59,-8.32,-8.32,-8.19,-8.65,-8.42,-8.36,-8.10,...,167457.0,74079.0,15460.0,17484.0,11350.0,12145.0,38281.0,10831.0,1,0
4,2021-01-04 10:16:00+00:00,0.22,0.50,0.23,0.21,0.23,0.23,0.15,0.01,-0.31,...,1588.0,1805.0,5776.0,6845.0,880.0,5369.0,3481.0,840.0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,-1.82,-1.94,-1.85,-1.85,-1.85,-1.82,-1.82,-1.94,-1.98,...,8934.0,19058.0,27278.0,14587.0,7554.0,6315.0,5149.0,2479.0,1,1
8652,2023-12-28 14:53:00+00:00,-0.08,0.64,0.68,0.62,-0.01,0.02,0.00,0.00,-0.01,...,82.0,2001.0,210.0,32.0,237.0,70.0,70.0,70.0,1,1
8653,2023-12-28 14:56:00+00:00,0.21,0.41,0.59,0.58,0.58,0.56,0.52,0.45,0.45,...,154.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0,1
8654,2023-12-28 14:56:00+00:00,0.21,0.41,0.59,0.58,0.58,0.56,0.52,0.45,0.45,...,154.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,0,1


In [3]:
for m in range(1, 11):
    df_total[f"delta_car_{m}m"] = df_total[f"abn_ret_{m}m"] - df_total[f"abn_ret_minus_{m}m"]
    df_total[f"post_car_{m}m"] = df_total[f"abn_ret_{m}m"]


In [4]:
df_total

,event_ts,abn_ret_minus_10m,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,...,delta_car_6m,post_car_6m,delta_car_7m,post_car_7m,delta_car_8m,post_car_8m,delta_car_9m,post_car_9m,delta_car_10m,post_car_10m
0,2021-01-04 10:08:00+00:00,-2.14,-2.14,-1.20,-1.32,-1.93,-2.38,-2.05,-2.45,-1.76,...,1.70,-0.23,1.08,-0.24,0.53,-0.67,1.80,-0.34,1.69,-0.45
1,2021-01-04 10:08:00+00:00,NaN,NaN,-1.45,-1.64,-1.44,-2.24,-2.49,-2.69,-1.96,...,0.20,-1.24,0.02,-1.62,0.08,-1.37,NaN,-1.04,NaN,-1.91
2,2021-01-04 10:08:00+00:00,-2.14,-2.14,-1.20,-1.32,-1.93,-2.38,-2.05,-2.45,-1.76,...,1.70,-0.23,1.08,-0.24,0.53,-0.67,1.80,-0.34,1.69,-0.45
3,2021-01-04 10:14:00+00:00,-8.40,-8.59,-8.32,-8.32,-8.19,-8.65,-8.42,-8.36,-8.10,...,4.68,-3.51,4.68,-3.64,4.15,-4.17,3.89,-4.70,3.50,-4.90
4,2021-01-04 10:16:00+00:00,0.22,0.50,0.23,0.21,0.23,0.23,0.15,0.01,-0.31,...,-0.47,-0.24,-0.27,-0.06,-0.47,-0.24,-1.00,-0.50,-0.19,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,-1.82,-1.94,-1.85,-1.85,-1.85,-1.82,-1.82,-1.94,-1.98,...,9.45,7.60,9.78,7.93,9.11,7.26,9.72,7.78,9.18,7.36
8652,2023-12-28 14:53:00+00:00,-0.08,0.64,0.68,0.62,-0.01,0.02,0.00,0.00,-0.01,...,0.22,0.21,-0.39,0.23,-0.41,0.27,-0.37,0.27,0.32,0.24
8653,2023-12-28 14:56:00+00:00,0.21,0.41,0.59,0.58,0.58,0.56,0.52,0.45,0.45,...,-0.10,0.48,-0.13,0.45,-0.14,0.45,0.01,0.42,0.21,0.42
8654,2023-12-28 14:56:00+00:00,0.21,0.41,0.59,0.58,0.58,0.56,0.52,0.45,0.45,...,-0.10,0.48,-0.13,0.45,-0.14,0.45,0.01,0.42,0.21,0.42


In [5]:
for m in range(1, 11):
    minus_cols = [f"volume_minus_{x}m" for x in range(m, 0, -1)]
    df_total[f"cum_volume_minus_{m}m"] = df_total[minus_cols].sum(axis=1)

# 누적 거래량 (cum_volume_1m ~ cum_volume_10m)
for m in range(1, 11):
    plus_cols = [f"volume_{x}m" for x in range(1, m + 1)]
    df_total[f"cum_volume_{m}m"] = df_total[plus_cols].sum(axis=1)
    

# volume으로 시작하는 컬럼 모두 drop
volume_cols_to_drop = [col for col in df_total.columns if col.startswith("volume")]
df_total = df_total.drop(columns=volume_cols_to_drop)


In [6]:
df_total.filter(regex="^cum_").head()

,cum_volume_minus_1m,cum_volume_minus_2m,cum_volume_minus_3m,cum_volume_minus_4m,cum_volume_minus_5m,cum_volume_minus_6m,cum_volume_minus_7m,cum_volume_minus_8m,cum_volume_minus_9m,cum_volume_minus_10m,cum_volume_1m,cum_volume_2m,cum_volume_3m,cum_volume_4m,cum_volume_5m,cum_volume_6m,cum_volume_7m,cum_volume_8m,cum_volume_9m,cum_volume_10m
0,175.0,2129.0,4899.0,5995.0,12027.0,14258.0,20333.0,25327.0,25347.0,25367.0,16549.0,27978.0,28716.0,31797.0,38441.0,41619.0,42678.0,44051.0,45295.0,46752.0
1,678.0,3414.0,4754.0,9270.0,11428.0,12870.0,13198.0,18159.0,18159.0,18159.0,19277.0,31427.0,42937.0,45893.0,49644.0,51426.0,55939.0,56276.0,56310.0,58014.0
2,175.0,2129.0,4899.0,5995.0,12027.0,14258.0,20333.0,25327.0,25347.0,25367.0,16549.0,27978.0,28716.0,31797.0,38441.0,41619.0,42678.0,44051.0,45295.0,46752.0
3,549.0,557.0,658.0,811.0,1540.0,2617.0,3382.0,7552.0,7629.0,14141.0,172251.0,344502.0,511959.0,586038.0,601498.0,618982.0,630332.0,642477.0,680758.0,691589.0
4,18708.0,31931.0,38483.0,41085.0,45101.0,62675.0,71568.0,78630.0,83825.0,98085.0,6055.0,11175.0,12763.0,14568.0,20344.0,27189.0,28069.0,33438.0,36919.0,37759.0


In [7]:
import numpy as np

# delta_cum_volume_{m}m = (cum_volume_{m}m - cum_volume_minus_{m}m) / cum_volume_minus_{m}
for m in range(1, 11):
    denom = 1 + df_total[f"cum_volume_minus_{m}m"]
    num = 1 + df_total[f"cum_volume_{m}m"]
    df_total[f"delta_cum_volume_{m}m"] = np.log(num / denom)

# min-max scaling for each delta_cum_volume_{m}m to [-1, 1]
for m in range(1, 11):
    col = f"delta_cum_volume_{m}m"
    x = df_total[col].values
    x_min = np.nanmin(x)
    x_max = np.nanmax(x)
    df_total[col] = 2 * (x - x_min) / (x_max - x_min) - 1

# 누적 거래량으로 시작하는 컬럼 모두 drop
cum_cols_to_drop = [col for col in df_total.columns if col.startswith("cum_volume_")]
df_total = df_total.drop(columns=cum_cols_to_drop)
df_total.filter(regex="^delta_cum_").head()


,delta_cum_volume_1m,delta_cum_volume_2m,delta_cum_volume_3m,delta_cum_volume_4m,delta_cum_volume_5m,delta_cum_volume_6m,delta_cum_volume_7m,delta_cum_volume_8m,delta_cum_volume_9m,delta_cum_volume_10m
0,0.333486,0.161402,0.088946,0.085629,0.044568,0.036476,0.011251,-0.008535,-0.009986,-0.007279
1,0.232936,0.132261,0.123457,0.080221,0.068455,0.060749,0.065219,0.035202,0.031363,0.033821
2,0.333486,0.161402,0.088946,0.085629,0.044568,0.036476,0.011251,-0.008535,-0.009986,-0.007279
3,0.434504,0.476734,0.479074,0.471461,0.418649,0.376194,0.355772,0.286021,0.283405,0.237671
4,-0.142715,-0.135503,-0.140313,-0.126794,-0.107865,-0.110893,-0.117568,-0.115191,-0.115059,-0.124281


In [8]:
df_total.filter(regex="^delta_cum_").describe()


,delta_cum_volume_1m,delta_cum_volume_2m,delta_cum_volume_3m,delta_cum_volume_4m,delta_cum_volume_5m,delta_cum_volume_6m,delta_cum_volume_7m,delta_cum_volume_8m,delta_cum_volume_9m,delta_cum_volume_10m
count,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000
mean,0.163739,0.136650,0.116684,0.110005,0.098784,0.089216,0.082213,0.070207,0.061515,0.057974
std,0.246935,0.217868,0.200190,0.188526,0.180285,0.173094,0.167554,0.161789,0.157080,0.153931
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-0.021705,-0.017887,-0.022577,-0.018487,-0.022234,-0.024684,-0.025981,-0.033382,-0.038513,-0.038890
50%,0.144107,0.111970,0.093918,0.086244,0.074949,0.066358,0.061280,0.049647,0.041583,0.039072
75%,0.329797,0.273464,0.238136,0.221723,0.203932,0.190321,0.178890,0.163284,0.150244,0.144150
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
df_total[df_total["period_dummy"] == 1].head()

,event_ts,abn_ret_minus_10m,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,...,delta_cum_volume_1m,delta_cum_volume_2m,delta_cum_volume_3m,delta_cum_volume_4m,delta_cum_volume_5m,delta_cum_volume_6m,delta_cum_volume_7m,delta_cum_volume_8m,delta_cum_volume_9m,delta_cum_volume_10m
5072,2022-07-01 10:03:00+00:00,-1.80,-0.66,0.14,0.19,-0.19,-0.19,0.28,-0.14,-0.14,...,-0.197859,-0.124490,-0.068736,-0.113969,-0.110240,-0.064567,-0.066950,-0.054036,-0.055558,-0.070355
5073,2022-07-01 10:24:00+00:00,-1.57,-1.43,-1.43,-1.43,-1.28,-1.23,-1.19,-1.66,-1.66,...,0.239031,0.207561,0.179545,0.193374,0.191108,0.189815,0.149814,0.117653,0.101474,0.086810
5074,2022-07-01 10:26:00+00:00,0.10,-0.05,0.10,0.00,0.05,-0.14,-0.14,-0.14,-0.05,...,0.136659,0.130844,0.152217,0.140227,0.135902,0.137003,0.139673,0.065761,0.060644,0.069101
5075,2022-07-01 10:43:00+00:00,-0.89,-0.99,-1.11,-1.13,-1.03,-1.13,-1.18,-1.26,-1.34,...,-0.059102,-0.055807,-0.057294,-0.054096,-0.076047,-0.077524,-0.077126,-0.071888,-0.063153,-0.066351
5076,2022-07-01 11:03:00+00:00,-0.21,-0.30,-0.35,-0.28,-0.04,-0.04,-0.14,0.04,-0.06,...,-0.077906,-0.082003,0.024876,0.059036,0.049761,0.044802,0.023395,0.014264,0.005169,-0.006780


In [ ]:
# df_total
# abn_ret만 써서 사용
# 10 min에서 하나라도 null 값 제외 (volume도 마찬가지)
# label: 1 > abn_ret argmax TTP 계산
# label: -1 > abn_ret argmin TTP 계산

#ln(TTP) = alpha + beta * telegram_dummy + gamma * min-max scale(ln(VolumeDiff)) + epsilon

# 회귀분석

### 📘 모형식

$$
\left| CAR_{post,i} - CAR_{pre,i} \right| = \alpha + \beta \cdot Telegram_i + \epsilon_i
$$

---

### 📊 변수 설명

**종속변수 (Dependent Variable)**  
이벤트 전후 CAR 변화폭의 절댓값:

$$
\left| CAR_{post,i} - CAR_{pre,i} \right|
$$


**설명변수 (Key Independent Variable)**  
텔레그램 도입 여부 변수:

$$
Telegram_i =
\begin{cases}
1, & \text{텔레그램 도입 이후 (After introduction)} \\
0, & \text{도입 이전 (Before introduction)}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{텔레그램 도입 이후와 이전 간 CAR 변화폭의 차이가 없다.}) \\
H_1 &: \beta < 0 \quad &(\text{텔레그램 도입 이후 CAR 변화폭이 감소했다.})
\end{aligned}
$$
---

### 📈 해석

이 회귀모형은 **텔레그램 도입 여부**가 이벤트 전후 누적초과수익률(CAR) 변화폭의 크기에 미치는 영향을 분석하기 위한 것입니다.  
β < 0 이고 통계적으로 유의하다면 귀무가설이 기각되어 공시 정보가 도입 이후 시장에 보다 신속하게 반영되어 정보 비효율성이 감소했음을 시사하며, 텔레그램 공시 알림 서비스가 시장 효율성 향상에 실질적으로 기여했음을 보여줍니다.


In [10]:
# import numpy as np
# import pandas as pd
# import statsmodels.api as sm

# # df_total이 이미 존재한다고 가정
# horizons = list[int](range(1, 11))
# rows = []

# for h in horizons:
#     post_col = f"abn_ret_{h}m"
#     pre_col  = f"abn_ret_minus_{h}m"
#     if post_col not in df_total.columns or pre_col not in df_total.columns:
#         raise KeyError(f"'{post_col}' 또는 '{pre_col}' 컬럼을 찾을 수 없습니다.")

#     y = (df_total[post_col] - df_total[pre_col]).abs()
#     X = sm.add_constant(df_total["period_dummy"], has_constant="add")

#     valid = ~(y.isna() | X.isna().any(axis=1))
#     y_ = y[valid]
#     X_ = X[valid]

#     model = sm.OLS(y_, X_).fit(cov_type="HC1")

#     beta = model.params.get("period_dummy", np.nan)
#     se   = model.bse.get("period_dummy", np.nan)
#     tval = model.tvalues.get("period_dummy", np.nan)
#     pval = model.pvalues.get("period_dummy", np.nan)

#     rows.append({
#         "horizon_min": h,
#         "beta": beta,
#         "std_err": se,
#         "t_value": tval,
#         "p_value": pval,
#         "sig_0.05": "*" if (not pd.isna(pval) and pval < 0.05) else "",
#         "n_obs": int(model.nobs),
#         "r_squared": model.rsquared
#     })

# result_table = pd.DataFrame(rows).sort_values("horizon_min").reset_index(drop=True)

# # ✅ p-value 포맷 조정 (지수표기 없이 소수점 8자리, 0은 "<1e-8"로 표시)
# result_table["p_value"] = result_table["p_value"].apply(
#     lambda x: f"{x:.12f}" if (x > 0 and not pd.isna(x)) else "<1e-8"
# )

# # 보기 좋게 출력
# display_cols = ["horizon_min", "beta", "std_err", "t_value", "p_value", "sig_0.05", "n_obs", "r_squared"]
# print(result_table[display_cols].to_string(index=False))

# 방향 정합성

### 📘 모형식 정의

$$
\text{logit}^{-1}(x) = \frac{1}{1 + e^{-x}}
$$

$$
Pr(hit_{i,t} = 1) = \text{logit}^{-1}(\alpha + \beta \cdot period\_dummy_i)
$$
---

### 📊 변수 설명


이벤트 *i*, 창 *t* 에 대해 Delta CAR, post CAR 2개의 지표를 사용

$$
t \in \{10,\,20,\,\dots,\,60\}
$$

$$
\Delta CAR_{i,t} = CAR_{post,i,t} - CAR_{pre,i,t}  
$$

$$
CAR_{post,i,t}
$$

$$
label\_sign_i =
\begin{cases}
+1, & \text{긍정 (1)} \\
0, & \text{중립 (0)} \\
-1, & \text{부정 (-1)} 
\end{cases}
$$



$$
hit_{i,t} =
\begin{cases}
1, & \text{if } \operatorname{sign}(\Delta CAR_{i,t}) = label\_sign_i \\
0, & \text{otherwise}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{도입 전후 차이 없음}) \\
H_1 &: \beta > 0 \quad &(\text{도입 후 정합률 상승})
\end{aligned}
$$

---

### 📈 해석

- delta CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 과잉반응 감소 / 즉시 반영 후 빠른 수렴 → 정보의 선반영 및 사전 확산이 강화 → 정보 효율성 향상”을 의미한다고 볼 수 있다.
- Post CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 이벤트 발표 이후의 정합률(정확도) 증가 → 시장 반응의 즉시 반영 강화 → 정보 효율성 향상”을 의미한다고 해석할 수 있다.


“중립 이벤트를 포함한 모델에서는 텔레그램 도입 이후 정합률이 유의하게 상승하였으나,
중립을 제외한 경우에는 그 효과가 약화되었다. 이는 정보 효율성 개선이
주로 가격 반응의 ‘적정성(overshooting 감소)’ 혹은 ‘비과잉 반응’ 영역에서 나타났음을 시사한다.
다시 말해, 시장이 새로운 정보를 과도하게 반응하지 않고 신속히 흡수하게 된 결과로 볼 수 있다.”

또는 요약형으로:

“효율성 개선은 방향성 예측력이 아니라, 불필요한 반응을 줄이는 형태로 나타났다.”

## 중립 포함

In [11]:
import sys
import os
sys.path.append('.')

from hypothesis_test_volume import (
    print_sample_summary_with_neutral,
    logistic_hit_delta_with_neutral_and_vol,
    logistic_hit_postCAR_with_neutral_and_vol,
    run_logistic_table
)

# === 전체 파이프라인 (중립 포함, 중립은 ΔCAR/POSTCAR 절대값 기준) ===

df = df_total.copy()
df["label_sign"] = df["label"]

df_full = print_sample_summary_with_neutral(df, "label_sign")
# 창 목록
windows = list(range(1, 11))

# ΔCAR + 거래량 공변량 (중립 포함)
results_delta_w_neutral_vol = run_logistic_table(
    logistic_hit_delta_with_neutral_and_vol,
    df_full,
    windows,
    label="ΔCAR + volume (neutral incl.)",
    covar_prefix="delta_cum_volume_",  # 기본값과 동일
    add_interaction=False,             # 상호작용 보려면 True
    robust_se=True
)

# postCAR + 거래량 공변량 (중립 포함)
results_post_w_neutral_vol = run_logistic_table(
    logistic_hit_postCAR_with_neutral_and_vol,
    df_full,
    windows,
    label="postCAR + volume (neutral incl.)",
    covar_prefix="delta_cum_volume_",
    add_interaction=False,
    robust_se=True
)



=== Sample summary (including neutral) ===
Total events:        8,656
Neutrals:            2,980
  - Positive (1):    4,906
  - Negative (-1):   770


--- ΔCAR + volume (neutral incl.) 기준 결과 ---
 t beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2  neutral_epsilon gamma_vol  std_vol    p_vol
 1   0.1100* 0.0468   2.350 0.018786       1.116     0.664    0.688     2.41   8656 0.0006  0.0000         0.525417    0.0629   0.0951 0.508428
 2   0.0982* 0.0469   2.094 0.036291       1.103     0.668    0.689     2.14   8656 0.0004 -0.0001         0.574956    0.0351   0.1097 0.748881
 3  0.1440** 0.0465   3.100 0.001938       1.155     0.648    0.680     3.21   8656 0.0017  0.0011         0.614614  0.3151**   0.1183 0.007731
 4  0.1325** 0.0462   2.868 0.004131       1.142     0.640    0.670     2.99   8656 0.0020  0.0014         0.602829 0.4258***   0.1244 0.000622
 5 0.1525*** 0.0460   3.317 0.000911       1.165     0.631    0.666     3.47   8656 0

## 중립 제거

In [12]:
from hypothesis_test_volume import (
    print_sample_summary,
    logistic_hit_delta_with_vol,
    logistic_hit_postCAR_with_vol,
    run_logistic_table
)

# === 전체 파이프라인 ===

df = df_total.copy()
df["label_sign"] = df["label"]
df_nn = print_sample_summary(df, "label_sign")
windows = list(range(1, 11))

# ΔCAR + volume (중립 제거)
results_delta_vol = run_logistic_table(
    logistic_hit_delta_with_vol,
    df_nn, windows,
    label="ΔCAR + volume (neutral removed)",
    covar_prefix="delta_cum_volume_",  # 이미 만드신 컬럼 접두사
    add_interaction=False,             # 필요 시 True
    robust_se=True
)

# postCAR + volume (중립 제거)
results_post_vol = run_logistic_table(
    logistic_hit_postCAR_with_vol,
    df_nn, windows,
    label="postCAR + volume (neutral removed)",
    covar_prefix="delta_cum_volume_",
    add_interaction=False,
    robust_se=True
)



=== Sample summary (neutral removed) ===
Total events:        8,656
Removed neutrals:    2,980
Used (non-neutral):  5,676
  - Positive (1):    4,906
  - Negative (-1):   770


--- ΔCAR + volume (neutral removed) 기준 결과 ---
 t beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2 gamma_vol  std_vol  p_vol
 1   -0.0727 0.0652  -1.114 0.265228       0.930     0.775    0.763    -1.22   5676 0.0523  0.0513 2.4319***   0.1462    0.0
 2   -0.0887 0.0646  -1.373 0.169759       0.915     0.773    0.758    -1.51   5676 0.0447  0.0437 2.5444***   0.1695    0.0
 3   -0.0276 0.0631  -0.437 0.662072       0.973     0.750    0.745    -0.49   5676 0.0485  0.0476 2.8805***   0.1847    0.0
 4   -0.0548 0.0620  -0.885 0.376156       0.947     0.738    0.728    -1.02   5676 0.0457  0.0448 2.9534***   0.1916    0.0
 5    0.0215 0.0613   0.350 0.726173       1.022     0.725    0.729     0.41   5676 0.0378  0.0369 2.7671***   0.1946    0.0
 6   -0.0289 0.0603  -0.479 

정보의 선반영(anticipation)·사전확산 강화
- 텔레그램 도입 이후, 공시 직전에 감성 방향으로 미리 움직이는 비율/크기가 커졌을 가능성
- 그래서 공시 순간의 post CAR 방향 정합은 높아지지만(β_post>0), “직전 대비 추가로 얼마나 같은 방향으로 더 가느냐”를 보는 ΔCAR 기준에서는 추가분이 줄어 **정합성↓**로 관측(β_Δ<0)

과잉반응 감소 / 즉시 반영 후 빠른 수렴
- 도입 전에는 공시 직후 같은 방향으로 과도하게 더 밀던 패턴이, 도입 후에는 이미 반영되어 추가 밀림이 줄어듦 → Δ 기준 정합성 감소(β_Δ<0).
- 하지만 “초기 방향” 자체는 더 정확해짐 → post 기준 정합성 증가(β_post>0).


도입 후 시장은 감성 방향으로 “빠르게 맞게 움직이지만”, 직전에 이미 반영된 탓에 공시 순간의 “추가” 같은 방향 밀림은 줄어든다. 이는 “정보의 더 빠른 확산·반영”이라는 효율성 개선 스토리와 일관됩니다.